In [2]:
import csv
import json
import requests
import numpy as np
import pandas as pd
import os

from surprise import BaselineOnly, Dataset, Reader
from surprise.model_selection import cross_validate

## Load data and map champion ID to Name and alphabetical index

In [3]:
filename = '../data/champion.json'
with open(filename, 'r') as f:
    champ_data = json.load(f)

# contains index of championId and name
id_champ_map = {}
cols = []
for i, c in enumerate(champ_data['data'].keys()):
    id_champ_map[int(champ_data['data'][c]['key'])] = {'champion': champ_data['data'][c]['id'], 'index': i}
    cols.append(champ_data['data'][c]['id'])

print(cols)
print(len(cols))
print(id_champ_map)

['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Briar', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'KSante', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'Milio', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Naafiri', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'Pyke', 'Qiy

## Save data in matrix format for analysis and df

In [14]:
filename = '../data/mastery_data.json'

with open(filename, 'r') as f:
    data = json.load(f)

names = []
r = np.zeros((len(data.keys()), len(cols)))

for i, key in enumerate(data.keys()):
    names.append([key])
    for j, entry in enumerate(data[key]):
        if j == 0:
            maximum = entry['championPoints']
        r[i][id_champ_map[entry['championId']]['index']] = entry['championPoints']/maximum

        fields = ['player_id']
print(r)
with open('../data/names.csv', 'w') as f:
    write = csv.writer(f)
     
    write.writerow(fields)
    write.writerows(names)

np.savetxt('../data/mastery.csv', r, delimiter=',', header=','.join(cols))

[[0.         0.         0.02915197 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.34750708 0.35768539 ... 0.         0.         0.        ]
 ...
 [0.         0.         1.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.90094475 0.         ... 0.         0.16045564 0.        ]]


In [10]:
def z(x):
    return 1 / (1 + np.exp(40*(0.3-x)))

print(z(0.4))

0.9996646498695336


## Save Data in surprise format to load into surprise

In [28]:
filename = '../data/mastery_data.json'

with open(filename, 'r') as f:
    data = json.load(f)

nameDict = {}
fields = ["user", "item", "rating"]

ratings = [] # list of items
r = np.zeros((len(data.keys()), len(cols)))

for i, key in enumerate(data.keys()):
    used = []
    if i <= 4500:
        for j, entry in enumerate(data[key]):
            if j == 0:
                # sorted in descending order
                maximum = entry['championPoints']
            score = entry['championPoints']/maximum
            item = id_champ_map[entry['championId']]['champion']
            ratings.append([key, item, score])
            used.append(item)

        for champ in cols:
            if champ not in used:
                ratings.append([key, champ, 0])
    else:
        continue
        
print(len(ratings))
with open('../data/train_data_surprise.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(ratings)

742665


In [17]:
df = pd.read_csv('../data/mastery.csv')
n = pd.read_csv('../data/names.csv')

s = df.sum().sort_values()

In [34]:
sel = df.loc[(df['Yasuo']==1)]
sel = sel.sum().sort_values()
print(sel.tail(20))

Lucian       21.460632
Riven        21.731619
Kaisa        22.170346
Pyke         22.938771
Darius       23.495518
Lux          24.648049
Sylas        26.490796
Vayne        27.614882
Jhin         27.689451
Ekko         28.259758
Katarina     29.704740
MasterYi     30.174990
LeeSin       32.635071
Kayn         33.965952
Irelia       34.190233
Ezreal       39.038227
Yone         40.243341
Akali        47.916226
Zed          62.517101
Yasuo       296.000000
dtype: float64
